In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, relationship

In [24]:
# --- Create SQLite In-Memory Database ---
DATABASE_URL = "sqlite:///:memory:"  # In-memory SQLite database
engine = create_engine(DATABASE_URL, echo=False) # true if you want logging
# Create a Base class for SQLAlchemy ORM
Base = declarative_base()

In [ ]:
# --- Define Employee Table ---
class Employee(Base):
    __tablename__ = 'employees'
    id = Column(Integer, primary_key=True, autoincrement=True)
    first = Column(String, nullable=False)
    last = Column(String, nullable=False)
    pay = Column(Integer)

    # Relationship to Purchases
    purchases = relationship("Purchase", back_populates="employee")

    def __repr__(self):
        return f"<Employee(id={self.id}, first='{self.first}', last='{self.last}', pay={self.pay})>"

# --- Define Purchases Table ---
class Purchase(Base):
    __tablename__ = 'purchases'
    id = Column(Integer, primary_key=True, autoincrement=True)
    employee_id = Column(Integer, ForeignKey('employees.id'), nullable=False)
    item = Column(String, nullable=False)
    amount = Column(Integer, nullable=False)
    
    # Relationship to Employee
    employee = relationship("Employee", back_populates="purchases")
    
    def __repr__(self):
        return f"<Purchase(id={self.id}, employee_id={self.employee_id}, item='{self.item}', amount={self.amount})>"


In [ ]:
# from pandasb

In [ ]:
# --- Create Tables ---
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# --- Insert Employees ---
emp_1 = Employee(first='John', last='Doe', pay=80000)
emp_2 = Employee(first='Jane', last='Doe', pay=90000)
session.add_all([emp_1, emp_2])
session.commit()


In [ ]:
# --- Insert Purchases ---
purchase_1 = Purchase(employee_id=emp_1.id, item='Laptop', amount=1200)
purchase_2 = Purchase(employee_id=emp_2.id, item='Monitor', amount=300)
session.add_all([purchase_1, purchase_2])
session.commit()

# --- Query: Get Employees by Last Name and Return as DataFrame ---
def get_emps_by_name(lastname):
    results = session.query(Employee).filter(Employee.last == lastname).all()
    emp_data = [{"id": emp.id, "first": emp.first, "last": emp.last, "pay": emp.pay} for emp in results]
    return pd.DataFrame(emp_data)

print("\nEmployees DataFrame:")
print(get_emps_by_name('Doe'))

# --- Query: Join Employees and Purchases ---
def get_employee_purchases():
    query = session.query(Employee.first, Employee.last, Purchase.item, Purchase.amount).\
        join(Purchase, Employee.id == Purchase.employee_id).all()
    
    purchase_data = [{"first": emp, "last": last, "item": item, "amount": amount} for emp, last, item, amount in query]
    return pd.DataFrame(purchase_data)

print("\nEmployee Purchases:")
print(get_employee_purchases())

# --- Close Session ---
session.close()

In [28]:
# --- Query: Get Employees by Last Name and Return as DataFrame ---
def get_emps_by_name(lastname):
    results = session.query(Employee).filter(Employee.last == lastname).all()
    
    # Convert to a list of dictionaries
    emp_data = [{"first": emp.first, "last": emp.last, "pay": emp.pay} for emp in results]
    
    # Convert to DataFrame
    return pd.DataFrame(emp_data)

# Fetch employees as a DataFrame
df_emps = get_emps_by_name('Doe')
print("\nEmployees DataFrame:\n", df_emps)



Employees DataFrame:
   first last    pay
0  John  Doe  80000
1  Jane  Doe  90000


In [29]:
# --- Update Employee Pay ---
def update_pay(emp, new_pay):
    emp.pay = new_pay
    session.commit()

update_pay(emp_2, 95000)

In [30]:
# --- Remove an Employee ---
def remove_emp(emp):
    session.delete(emp)
    session.commit()

remove_emp(emp_1)

In [32]:
# Verify Updates
emps = get_emps_by_name('Doe')
print("\nUpdated Employees:\n", emps)


Updated Employees:
   first last    pay
0  Jane  Doe  95000


In [ ]:
# --- Load and Execute SQL Query from File ---
query_file = "sql_queries/Q1.txt"
try:
    with open(query_file, "r") as file:
        query = file.read().strip()

    print(f"\nExecuting Query from {query_file}:\n{query}")
    df_query_result = pd.read_sql_query(query, con=engine)
    print("\nQuery Result:")
    print(df_query_result)

except FileNotFoundError:
    print(f"\nError: Query file '{query_file}' not found.")

In [ ]:
# --- Append Data from CSV File ---
csv_file = "customers.csv"
try:
    df_csv = pd.read_csv(csv_file)

    # Rename columns to match database schema
    df_csv.rename(columns={
        'Customer Name': 'name',
        'Customer Phone': 'phone_number'
    }, inplace=True)

    # Append to the database
    df_csv.to_sql(name='customers', con=engine, if_exists='append', index=False)
    print(f"\nData from {csv_file} appended to 'customers' table.")

except FileNotFoundError:
    print(f"\nError: CSV file '{csv_file}' not found.")

# Close session
session.close()
